In [13]:
import torch
import tensorflow as tf
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from Single_Block_Fortnite import EmbeddingNN, AttentionNN, NormNN, FFN

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open("C:/Users/PC/Desktop/Important Data/Shakespeare_Data.txt", "r") as f:
    text = f.read()  
    chars = set(text)
    vocab_size = len(chars)
print(chars, vocab_size)

decode_dict = { i : chars for i , chars in enumerate(chars)}
encode_dict = { chars : i for i , chars in enumerate(chars)}

encode = lambda text : [encode_dict[i] for i in text]
decode = lambda text : ''.join([decode_dict[i] for i in text])

encoded_text = encode(text)

{'z', ',', 'j', 'E', 'b', 'i', '$', 'M', ' ', 'f', 'V', 'U', 's', 'Z', '3', 'H', 'B', 'n', 'r', 'G', 'T', 'Q', 'Y', 'x', 'o', 'I', 'd', 'm', 'q', '-', 'u', 'D', 'y', "'", 'c', 'a', 'l', 'v', 'X', 'L', '!', 't', 'K', 'F', 'k', 'e', '.', 'w', 'R', 'C', 'p', 'h', ';', 'S', 'O', '\n', 'P', 'W', ':', 'A', '?', 'N', 'J', 'g', '&'} 65


In [15]:
batch_size = 64
hidden_dim = 128
max_sequence_length = 512
num_of_head = 8

In [16]:
class TransformerBlock(nn.Module):
    def __init__(self, hidden_dim, num_heads):
        super().__init__()

        self.attention = AttentionNN(hidden_dim, num_heads)
        self.ffn = FFN(hidden_dim)
        self.norm1 = NormNN(hidden_dim)
        self.norm2 = NormNN(hidden_dim)

    def forward(self, X):
        attention_output = self.attention(X)
        X = X + attention_output

        pog_X = self.norm1(X)

        pogger_X = self.ffn(pog_X) + pog_X

        poggers_X = self.norm2(pogger_X)

        return poggers_X

        

In [17]:
embedding_model = EmbeddingNN(vocab_size, hidden_dim, max_sequence_length).to(device)

data = torch.tensor(encoded_text , dtype = torch.long).to(device)


seq_len = 64
for i in range(0, len(X) - seq_len, seq_len):
    batch_x = data[i:i+seq_len].unsqueeze(0)
    batch_x = batch_x.repeat(batch_size, 1)

    pos = torch.arange(seq_len, device=device).unsqueeze(0).repeat(batch_size, 1)


    combined_embeddings = embedding_model(batch_x, pos)


In [18]:
class Transformer(nn.Module):
    def __init__(self,vocab_size , hidden_dim,num_heads, N):
        super().__init__()

        self.blocks = nn.ModuleList(TransformerBlock(hidden_dim, num_heads) for i in range(N))
        self.linear = nn.Linear(hidden_dim,vocab_size)

    def forward(self, X):
        for block in self.blocks:
            X = block(X)
        X = self.linear(X)

        return X